In [2]:
from scipy.special import lambertw
import numpy as np


def calc_vd(VCC):
    I0 = 1e-12
    n = 1
    VT = 0.026
    R = 100

    argumento = (R * I0) / (n * VT)
    exponente = (VCC + R * I0) / (n * VT)

    W0 = lambertw(argumento * np.exp(exponente)).real

    VD = VCC + (R * I0) - (n * VT * W0)

    return VD


In [9]:
import tensorflow as tf

# Definir la entrada y el valor objetivo (salida deseada) para el entrenamiento
X = tf.constant([[3], [6], [9]], dtype=tf.float32)
y = tf.constant([[calc_vd(3)], [calc_vd(6)], [calc_vd(9)]], dtype=tf.float32)

# Construir el modelo como una red neuronal secuencial
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(1,)),
        # Capa 1: Densa lineal sigmoide
        tf.keras.layers.Dense(
            3,
            activation="sigmoid",
            kernel_initializer=tf.constant_initializer([[0.05], [0.15], [-0.2]]),
            bias_initializer=tf.constant_initializer([0.23, -0.1, 0.17]),
        ),
        # Capa 2: Densa lineal relu
        tf.keras.layers.Dense(
            2,
            activation="relu",
            kernel_initializer=tf.constant_initializer(
                [[0.8, -0.6], [0.7, 0.9], [0.5, -0.6]]
            ),
            bias_initializer=tf.constant_initializer([0.45, -0.34]),
        ),
        # Capa 3: Densa lineal que produce un vector de salida
        tf.keras.layers.Dense(
            1,
            activation=None,
            kernel_initializer=tf.constant_initializer([[0.8], [0.5]]),
            bias_initializer=tf.constant_initializer([0.7]),
        ),
    ]
)

# Compilar el modelo con función de pérdida y optimizador
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss="mse", metrics=["mse"]
)

predicciones_iniciales = model(X).numpy()
predicciones_iniciales_rounded = tf.round(predicciones_iniciales * 10000) / 10000
error_inicial = tf.reduce_mean(tf.square(predicciones_iniciales - y)).numpy()

print("Predicciones iniciales:\n", predicciones_iniciales_rounded.numpy())
print("Error inicial (MSE): ", error_inicial, "\n")

# Entrenar el modelo
epochs = 1
history = model.fit(X, y, epochs=epochs, verbose=0)

# Mostrar la salida final del modelo y el error final redondeados
predicciones_finales = model(X)
predicciones_finales_rounded = tf.round(predicciones_finales * 10000) / 10000
error_final = tf.reduce_mean(tf.square(predicciones_finales - y)).numpy()

print("Predicciones finales (Lineal):\n", predicciones_finales_rounded.numpy())
print("Error final (MSE):", error_final)

# Definir nombres para los pesos y sesgos
nombres_pesos = ['Ws', 'Wu', 'Wy']
nombres_sesgos = ['bs', 'bu', 'by']

# Ver los pesos actualizados después de una época de entrenamiento
for i, layer in enumerate(model.layers):
    if hasattr(layer, 'get_weights'):  # Comprobar que la capa tiene pesos
        weights, biases = layer.get_weights()  # Obtener pesos y sesgos
        print(f"Pesos {nombres_pesos[i]}:\n", weights)
        print(f"Sesgos {nombres_sesgos[i]}:\n", biases)

Predicciones iniciales:
 [[1.9262]
 [1.9545]
 [1.9856]]
Error inicial (MSE):  1.7328434 

Predicciones finales (Lineal):
 [[1.7856]
 [1.7949]
 [1.8115]]
Error final (MSE): 1.3414707
Pesos Ws:
 [[ 0.02672244  0.13242216 -0.21094187]]
Sesgos bs:
 [ 0.22608304 -0.10309005  0.16800411]
Pesos Wu:
 [[ 0.78674686 -0.6       ]
 [ 0.6855674   0.9       ]
 [ 0.49425203 -0.6       ]]
Sesgos bu:
 [ 0.42893875 -0.34      ]
Pesos Wy:
 [[0.7586789]
 [0.5      ]]
Sesgos by:
 [0.67367345]
